# Work in progress!

### Alice creates her key, encodes it with her bases, and tries to send it to Bob

In [1]:
from random import choices
import cirq

num_bits = 10

encode_gates = {0: cirq.I, 1: cirq.X} # Gates that produce the 0 and 1 quantums states, which represents the keys
basis_gates = {"Z": cirq.I, "X": cirq.H} # Gates that will create the correct qubit encryption

message = "Quantum Computing"

alice_key = choices([0, 1], k = num_bits)
print("Alice Key: ", alice_key)
alice_bases = choices(['Z', 'X'], k = num_bits)
print("Alice Bases: ", alice_bases)

alice_circuit = cirq.Circuit()
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

for bit in range(num_bits):

 encode_value = alice_key[bit]
 encode_gate = encode_gates[encode_value]

 basis_value = alice_bases[bit]
 basis_gate = basis_gates[basis_value]

 qubit = qubits[bit]
 alice_circuit.append(encode_gate(qubit))
 alice_circuit.append(basis_gate(qubit))
print(alice_circuit)

Alice Key:  [1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
Alice Bases:  ['X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X']
q0: ───X───H───

q1: ───I───I───

q2: ───I───H───

q3: ───I───H───

q4: ───X───H───

q5: ───I───H───

q6: ───X───I───

q7: ───I───H───

q8: ───X───H───

q9: ───X───H───


### Add noise to our model
* QKD is meant to be done over large distances through noisy mediums such as the atmosphere. Thus, an accurate model of QKD must include noise.
* For this model, I will use depolarizing noise, where X, Y, and Z gates are applied randomly. This depolarizing noise will be applied to a specific moment in the circuit - specifically, the moment when Alice sends her qubits to Bob.

In [4]:
noisy_circuit = cirq.Circuit()
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

noise = cirq.depolarize(0.05)
noisy_circuit.append(noise.on_each(qubits))

print(noisy_circuit)

q0: ───D(0.05)───

q1: ───D(0.05)───

q2: ───D(0.05)───

q3: ───D(0.05)───

q4: ───D(0.05)───

q5: ───D(0.05)───

q6: ───D(0.05)───

q7: ───D(0.05)───

q8: ───D(0.05)───

q9: ───D(0.05)───


### The measurement attack:
* Eve recieves Alice's qubits and measures all of them. Using the measurements, she infers the states of Alice's qubits, and sends her guesses over to Bob. In my scenario, Eve will simply send her measurements over to Bob. However, half of these guesses will be wrong because of the probabilitic nature of qubits in superposition. Thus, Bob will easily be able to detect a measurement attack.

In [5]:
eve_circuit = cirq.Circuit()
eve_circuit.append(cirq.measure(qubits, key="eve intercept"))# COMPLETE THIS CODE
eve_circuit

q0: ───M('eve intercept')───
       │
q1: ───M────────────────────
       │
q2: ───M────────────────────
       │
q3: ───M────────────────────
       │
q4: ───M────────────────────
       │
q5: ───M────────────────────
       │
q6: ───M────────────────────
       │
q7: ───M────────────────────
       │
q8: ───M────────────────────
       │
q9: ───M────────────────────

In [7]:
eve_intercept_circuit = alice_circuit + noisy_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve intercept'][0]

print(eve_intercept_circuit)

print('\nEve\'s initial key: ', eve_key)

q0: ───X───H───D(0.05)───M('eve intercept')───
                         │
q1: ───I───I───D(0.05)───M────────────────────
                         │
q2: ───I───H───D(0.05)───M────────────────────
                         │
q3: ───I───H───D(0.05)───M────────────────────
                         │
q4: ───X───H───D(0.05)───M────────────────────
                         │
q5: ───I───H───D(0.05)───M────────────────────
                         │
q6: ───X───I───D(0.05)───M────────────────────
                         │
q7: ───I───H───D(0.05)───M────────────────────
                         │
q8: ───X───H───D(0.05)───M────────────────────
                         │
q9: ───X───H───D(0.05)───M────────────────────

Eve's initial key:  [1 0 1 1 1 0 0 1 0 0]


#### Even then sends her measured qubits to Bob in a noisy environment. 
* The same depolarizing noise is applied.
#### Bob applies the normal procedure to get his key

In [8]:
bob_bases = choices(["Z", "X"], k=num_bits)# COMPLETE THIS CODE


bob_circuit = cirq.Circuit()

for bit in range(num_bits):# COMPLETE THIS CODE

  basis_value = bob_bases[bit]# COMPLETE THIS CODE
  basis_gate = basis_gates[basis_value]# COMPLETE THIS CODE

  qubit = qubits[bit]# COMPLETE THIS CODE
  bob_circuit.append(basis_gate(qubit))# COMPLETE THIS CODE

bob_circuit.append(cirq.measure(qubits, key="bob key"))# COMPLETE THIS CODE

print(bob_circuit)

q0: ───I───M('bob key')───
           │
q1: ───I───M──────────────
           │
q2: ───H───M──────────────
           │
q3: ───H───M──────────────
           │
q4: ───I───M──────────────
           │
q5: ───H───M──────────────
           │
q6: ───H───M──────────────
           │
q7: ───H───M──────────────
           │
q8: ───H───M──────────────
           │
q9: ───I───M──────────────


In [9]:
bb84_circuit = alice_circuit + noisy_circuit + eve_intercept_circuit + noisy_circuit + bob_circuit
print(bb84_circuit)

q0: ───X───H───D(0.05)───X───H───D(0.05)───M('eve intercept')───D(0.05)───I───M('bob key')───
                                           │                                  │
q1: ───I───I───D(0.05)───I───I───D(0.05)───M────────────────────D(0.05)───I───M──────────────
                                           │                                  │
q2: ───I───H───D(0.05)───I───H───D(0.05)───M────────────────────D(0.05)───H───M──────────────
                                           │                                  │
q3: ───I───H───D(0.05)───I───H───D(0.05)───M────────────────────D(0.05)───H───M──────────────
                                           │                                  │
q4: ───X───H───D(0.05)───X───H───D(0.05)───M────────────────────D(0.05)───I───M──────────────
                                           │                                  │
q5: ───I───H───D(0.05)───I───H───D(0.05)───M────────────────────D(0.05)───H───M──────────────
                                    

In [10]:
sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [1 0 1 0 1 0 1 1 0 1]


### Bob and Alice attempt to create a shared final key. 
* This final key should not match, both because of Eve's interception and because of the noise.

In [11]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):
    if alice_bases[bit] == bob_bases[bit]:
        final_alice_key.append(alice_key[bit])
        final_bob_key.append(bob_key[bit])
print("Final Alice key: ", final_alice_key)
print("Final Bob key: ", final_bob_key)

Final Alice key:  [0, 0, 0, 0, 0, 1]
Final Bob key:  [0, 1, 0, 0, 1, 0]


* **Statistically, Alice's key and Bob's key should only match 50%-ish**

In [12]:
total_agree = 0

for i in range(len(final_bob_key)):
    if final_bob_key[i]==final_alice_key[i]:
        total_agree += 1

print("Percentage of Alice's final key that matches with Bob's final key: ", (total_agree/len(final_alice_key)))
    

Percentage of Alice's final key that matches with Bob's final key:  0.5
